# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 6 2022 1:06PM,238888,18,CLRSPC-21048111,"ClearSpec, LLC",Released
1,Jun 6 2022 1:03PM,238886,12,HH6199,Hush Hush,Released
2,Jun 6 2022 1:02PM,238887,12,HH-34920,Hush Hush,Released
3,Jun 6 2022 1:02PM,238887,12,HH-34923,Hush Hush,Released
4,Jun 6 2022 1:02PM,238887,12,HH-34928,Hush Hush,Released
5,Jun 6 2022 12:57PM,238885,10,Enova-10084,Emerginnova,Released
6,Jun 6 2022 12:56PM,238884,10,Enova-10084-1,Emerginnova,Released
7,Jun 6 2022 12:53PM,238883,18,CLRSPC-30598325,"ClearSpec, LLC",Released
8,Jun 6 2022 12:36PM,238881,12,HH6197,Hush Hush,Released
9,Jun 6 2022 12:23PM,238880,19,SD06072022,"SD Head USA, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
26,238884,Released,1
27,238885,Released,1
28,238886,Released,1
29,238887,Released,3
30,238888,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
238884,NaN,NaN,1.0
238885,NaN,NaN,1.0
238886,NaN,NaN,1.0
238887,NaN,NaN,3.0
238888,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238820,5.0,4.0,4.0
238832,0.0,0.0,39.0
238836,0.0,0.0,51.0
238838,0.0,0.0,1.0
238842,0.0,0.0,61.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238820,5,4,4
238832,0,0,39
238836,0,0,51
238838,0,0,1
238842,0,0,61


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238820,5,4,4
1,238832,0,0,39
2,238836,0,0,51
3,238838,0,0,1
4,238842,0,0,61


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238820,5,4,4
1,238832,,,39
2,238836,,,51
3,238838,,,1
4,238842,,,61


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 6 2022 1:06PM,238888,18,"ClearSpec, LLC"
1,Jun 6 2022 1:03PM,238886,12,Hush Hush
2,Jun 6 2022 1:02PM,238887,12,Hush Hush
5,Jun 6 2022 12:57PM,238885,10,Emerginnova
6,Jun 6 2022 12:56PM,238884,10,Emerginnova
7,Jun 6 2022 12:53PM,238883,18,"ClearSpec, LLC"
8,Jun 6 2022 12:36PM,238881,12,Hush Hush
9,Jun 6 2022 12:23PM,238880,19,"SD Head USA, LLC"
10,Jun 6 2022 12:20PM,238879,20,"ACI Healthcare USA, Inc."
54,Jun 6 2022 12:18PM,238878,10,Eywa Pharma Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 6 2022 1:06PM,238888,18,"ClearSpec, LLC",,,1
1,Jun 6 2022 1:03PM,238886,12,Hush Hush,,,1
2,Jun 6 2022 1:02PM,238887,12,Hush Hush,,,3
3,Jun 6 2022 12:57PM,238885,10,Emerginnova,,,1
4,Jun 6 2022 12:56PM,238884,10,Emerginnova,,,1
5,Jun 6 2022 12:53PM,238883,18,"ClearSpec, LLC",,,1
6,Jun 6 2022 12:36PM,238881,12,Hush Hush,,,1
7,Jun 6 2022 12:23PM,238880,19,"SD Head USA, LLC",,,1
8,Jun 6 2022 12:20PM,238879,20,"ACI Healthcare USA, Inc.",,,44
9,Jun 6 2022 12:18PM,238878,10,Eywa Pharma Inc.,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 6 2022 1:06PM,238888,18,"ClearSpec, LLC",1,,
1,Jun 6 2022 1:03PM,238886,12,Hush Hush,1,,
2,Jun 6 2022 1:02PM,238887,12,Hush Hush,3,,
3,Jun 6 2022 12:57PM,238885,10,Emerginnova,1,,
4,Jun 6 2022 12:56PM,238884,10,Emerginnova,1,,
5,Jun 6 2022 12:53PM,238883,18,"ClearSpec, LLC",1,,
6,Jun 6 2022 12:36PM,238881,12,Hush Hush,1,,
7,Jun 6 2022 12:23PM,238880,19,"SD Head USA, LLC",1,,
8,Jun 6 2022 12:20PM,238879,20,"ACI Healthcare USA, Inc.",44,,
9,Jun 6 2022 12:18PM,238878,10,Eywa Pharma Inc.,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 6 2022 1:06PM,238888,18,"ClearSpec, LLC",1,,
1,Jun 6 2022 1:03PM,238886,12,Hush Hush,1,,
2,Jun 6 2022 1:02PM,238887,12,Hush Hush,3,,
3,Jun 6 2022 12:57PM,238885,10,Emerginnova,1,,
4,Jun 6 2022 12:56PM,238884,10,Emerginnova,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 6 2022 1:06PM,238888,18,"ClearSpec, LLC",1.0,NaN,NaN
1,Jun 6 2022 1:03PM,238886,12,Hush Hush,1.0,NaN,NaN
2,Jun 6 2022 1:02PM,238887,12,Hush Hush,3.0,NaN,NaN
3,Jun 6 2022 12:57PM,238885,10,Emerginnova,1.0,NaN,NaN
4,Jun 6 2022 12:56PM,238884,10,Emerginnova,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 6 2022 1:06PM,238888,18,"ClearSpec, LLC",1.0,0.0,0.0
1,Jun 6 2022 1:03PM,238886,12,Hush Hush,1.0,0.0,0.0
2,Jun 6 2022 1:02PM,238887,12,Hush Hush,3.0,0.0,0.0
3,Jun 6 2022 12:57PM,238885,10,Emerginnova,1.0,0.0,0.0
4,Jun 6 2022 12:56PM,238884,10,Emerginnova,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2149740,179.0,14.0,0.0
12,716654,5.0,0.0,0.0
15,238852,1.0,1.0,0.0
17,238873,1.0,0.0,0.0
18,477771,2.0,0.0,0.0
19,1194296,17.0,5.0,5.0
20,955448,111.0,1.0,0.0
22,238869,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2149740,179.0,14.0,0.0
1,12,716654,5.0,0.0,0.0
2,15,238852,1.0,1.0,0.0
3,17,238873,1.0,0.0,0.0
4,18,477771,2.0,0.0,0.0
5,19,1194296,17.0,5.0,5.0
6,20,955448,111.0,1.0,0.0
7,22,238869,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,179.0,14.0,0.0
1,12,5.0,0.0,0.0
2,15,1.0,1.0,0.0
3,17,1.0,0.0,0.0
4,18,2.0,0.0,0.0
5,19,17.0,5.0,5.0
6,20,111.0,1.0,0.0
7,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,179.0
1,12,Released,5.0
2,15,Released,1.0
3,17,Released,1.0
4,18,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,17,18,19,20,22
Status,,,,,,,,
Completed,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0
Executing,14.0,0.0,1.0,0.0,0.0,5.0,1.0,0.0
Released,179.0,5.0,1.0,1.0,2.0,17.0,111.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,17,18,19,20,22
0,Completed,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0
1,Executing,14.0,0.0,1.0,0.0,0.0,5.0,1.0,0.0
2,Released,179.0,5.0,1.0,1.0,2.0,17.0,111.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,17,18,19,20,22
0,Completed,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0
1,Executing,14.0,0.0,1.0,0.0,0.0,5.0,1.0,0.0
2,Released,179.0,5.0,1.0,1.0,2.0,17.0,111.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()